# CNN Training Components

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ishwar-git/Image-Classification/blob/main/CNN_Training_Components.ipynb)

## Overview
This notebook captures the **content outline** for training a Convolutional Neural Network (CNN).


## CNN Training Components

1. **Dataset Preparation**
   - Collect and preprocess data (resize, normalize, augment)
   - Split into train, validation, and test sets

2. **Model Initialization**
   - Define CNN architecture (Conv, ReLU, Pool, Dense)
   - Initialize weights (Xavier, He, random normal)

3. **Forward Propagation**
   - Input image passes through CNN layers
   - Convolution → Activation → Pooling → Flatten → Fully Connected → Output
   - Compute predicted output (ŷ)

4. **Loss Function**
   - Measures difference between predicted (ŷ) and true (y)
   - Common choices:
     - **Cross-Entropy Loss** (classification)
     - **Mean Squared Error (MSE)** (regression)

5. **Backward Propagation (Backprop through Convolution)**
   - Compute gradients of loss w.r.t. weights
   - Chain Rule propagates error from output → input layers
   - Update gradients using optimizer

6. **Optimizer**
   - Adjusts weights to minimize loss  
   - Examples:
     - **SGD (with Momentum)**
     - **Adam**
     - **RMSProp**
   - Update rule:  

     \[ w_{new} = w_{old} - \eta \times \frac{\partial L}{\partial w} \]

7. **Regularization Techniques**
   - **Dropout**: randomly deactivate neurons to avoid overfitting  
   - **Weight Decay (L2 Regularization)**  
   - **Batch Normalization**

8. **Hyperparameter Tuning**
   - Learning rate, batch size, number of epochs, optimizer choice
   - Grid search / random search / Bayesian optimization

9. **Evaluation & Validation**
   - Use validation/test set to measure performance  
   - Metrics:
     - Accuracy, Precision, Recall, F1-score  
     - Confusion matrix  
     - ROC curve

10. **Visualization & Monitoring**



### Flow Summary
```
Dataset → CNN Model → Forward Pass → Loss Calculation → Backpropagation 
→ Optimizer Update → Regularization → Evaluation → Visualization
```


In [14]:
# !pip install torch torchvision torchaudio  # (Uncomment if needed)

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device


device(type='cuda')

In [15]:
# 1) Dataset Preparation
transform_train = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
])
transform_test = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
])

# Example with CIFAR10 (auto-downloads if online; otherwise, replace with your dataset)
train_ds = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
test_ds  = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

train_loader = DataLoader(train_ds, batch_size=64, shuffle=True, num_workers=2)
test_loader  = DataLoader(test_ds, batch_size=128, shuffle=False, num_workers=2)
len(train_ds), len(test_ds)


Files already downloaded and verified
Files already downloaded and verified


(50000, 10000)

In [16]:
# 2) Model Initialization (simple CNN example)
class SmallCNN(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 16 * 16, 128),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3),
            nn.Linear(128, num_classes)
        )
    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

model = SmallCNN().to(device)
model


SmallCNN(
  (features): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=16384, out_features=128, bias=True)
    (2): ReLU(inplace=True)
    (3): Dropout(p=0.3, inplace=False)
    (4): Linear(in_features=128, out_features=10, bias=True)
  )
)

In [17]:
# 3) Loss Function & Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)  # L2 = weight_decay


In [18]:
# 4) Training Loop (Forward → Loss → Backprop → Optimizer Step)
def train_one_epoch(model, loader, optimizer, criterion):
    model.train()
    total_loss, total_correct, total = 0.0, 0, 0
    for x, y in loader:
        x, y = x.to(device), y.to(device)

        # Forward
        logits = model(x)
        loss = criterion(logits, y)

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Metrics
        total_loss += loss.item() * x.size(0)
        preds = logits.argmax(dim=1)
        total_correct += (preds == y).sum().item()
        total += x.size(0)
    return total_loss / total, total_correct / total

def evaluate(model, loader, criterion):
    model.eval()
    total_loss, total_correct, total = 0.0, 0, 0
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            logits = model(x)
            loss = criterion(logits, y)
            total_loss += loss.item() * x.size(0)
            preds = logits.argmax(dim=1)
            total_correct += (preds == y).sum().item()
            total += x.size(0)
    return total_loss / total, total_correct / total


In [19]:
# 5) Fit (demo: small number of epochs)
EPOCHS = 2
for epoch in range(EPOCHS):
    tr_loss, tr_acc = train_one_epoch(model, train_loader, optimizer, criterion)
    te_loss, te_acc = evaluate(model, test_loader, criterion)
    print(f"Epoch {epoch+1}/{EPOCHS} | train_loss={tr_loss:.4f} acc={tr_acc:.3f} | val_loss={te_loss:.4f} acc={te_acc:.3f}")


Epoch 1/2 | train_loss=1.4563 acc=0.476 | val_loss=1.1257 acc=0.600
Epoch 2/2 | train_loss=1.1326 acc=0.600 | val_loss=1.0173 acc=0.637


> **Note:** If this environment doesn't have internet access, CIFAR-10 won't auto-download. Replace the dataset block with your local data loader or run this notebook in your own environment where downloads are allowed.
